In [ ]:
!pip install transformers
!pip install datasets
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-160m-deduped"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep = '\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
IGNORED_PAD_IDX = -100
PAD_IDX

50278

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

files="/content/drive/MyDrive/AICUP/train_datas.csv"
datas = pd.read_csv(files)
print(len(datas))
#datas = datas[-3000:]
datas.head()

97334


,file,contents_start,contents_end,contents,labels
0,10,0,1,\t,PHI: NULL
1,10,1,25,Episode No: 09F016547J\t,IDNUM\t14\t24\t09F016547J\n
2,10,25,36,091016.NMT\t,MEDICALRECORD\t25\t35\t091016.NMT\n
3,10,36,37,\t,PHI: NULL
4,10,37,52,"SIZAR, HOWARD \t","PATIENT\t37\t50\tSIZAR, HOWARD\n"


In [ ]:
from torch.utils.data import DataLoader
import torch
import re

def modify_string(original_string):
    pattern = re.compile(r'(\w+)\t(\d+)\t(\d+)\t(.+)')
    modified_string = re.sub(pattern, r'\1\t\4', original_string)
    return modified_string

train_data = [bos + data['contents'] + sep + modify_string(data['labels']) + eos for _, data in datas.iterrows()]
print(train_data[-1])
print("============================================")
print(len(train_data))
#train_data = train_data[20000:50000]
#train_data = train_data[60000:80000]

def collate_batch(batch):

    texts = batch

    encoded_seq = tokenizer(texts, padding=True)

    indexed_tks = torch.tensor(encoded_seq['input_ids'])
    attention_mask = torch.tensor(encoded_seq['attention_mask'])
    encoded_label = torch.tensor(encoded_seq['input_ids'])

    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return indexed_tks, encoded_label, attention_mask

train_dataloader = DataLoader(train_data, batch_size=2, shuffle=False, collate_fn=collate_batch)
titer = iter(train_dataloader)
tks, labels, mask = next(titer)
print(tks.shape)
next(iter(titer))

<|endoftext|>Please contact our research team at (1111 2222) for collaboration inquiries.	

####

PHONE	1111 2222
<|END|>
97334
torch.Size([2, 25])


(tensor([[    0,  2693, 37524,    15,    47,  6229,   186, 50279, 19047, 25129,
          34181,  8242,   186,  2693, 37524,    15,    47,  6229,   187, 50277],
         [    0,   186, 50279,  6663,    42,    27,  5812, 50277, 50278, 50278,
          50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278]]),
 tensor([[    0,  2693, 37524,    15,    47,  6229,   186, 50279, 19047, 25129,
          34181,  8242,   186,  2693, 37524,    15,    47,  6229,   187, 50277],
         [    0,   186, 50279,  6663,    42,    27,  5812, 50277,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [ ]:
import random
BATCH_SIZE = 8

class BatchSampler():
    def __init__(self, data, batch_size):
        self.pooled_indice = []
        self.data = data
        self.batch_size = batch_size
        self.len = len(list(data))
    def __iter__(self):
        self.pooled_indices = []
        indices = [(index, len(data)) for index, data in enumerate(self.data)]
        random.shuffle(indices)
        for i in range(0, len(indices), BATCH_SIZE * 100):
            self.pooled_indices.extend(sorted(indices[i:i + BATCH_SIZE * 100], key=lambda x: x[1], reverse=True))
        self.pooled_indices = [x[0] for x in self.pooled_indices]

        for i in range(0, len(self.pooled_indices), BATCH_SIZE):
            yield self.pooled_indices[i:i + BATCH_SIZE]
    def __len__(self):
        return (self.len + self.batch_size - 1) // self.batch_size

bucket_train_dataloader = DataLoader(train_data, batch_sampler=BatchSampler(train_data, BATCH_SIZE),
                                     collate_fn=collate_batch, pin_memory=True)

In [ ]:
'''
for idx, batch in enumerate(bucket_train_dataloader):
    print(batch)
    print(batch[0].shape)
    print(batch[1].shape)
'''

'\nfor idx, batch in enumerate(bucket_train_dataloader):\n    print(batch)\n    print(batch[0].shape)\n    print(batch[1].shape)\n'

In [ ]:
import torch
from torch.optim import AdamW
model = AutoModelForCausalLM.from_pretrained(plm, revision='step3000')
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=5e-5)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
from tqdm import tqdm, trange

model.train()
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    predictions, true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

Epoch:  20%|██        | 1/5 [23:08<1:32:34, 1388.74s/it]

Average train loss: 1.2479228172522971


Epoch:  40%|████      | 2/5 [46:19<1:09:29, 1389.82s/it]

Average train loss: 1.0113534059743245


Epoch:  60%|██████    | 3/5 [1:09:35<46:25, 1392.77s/it]

Average train loss: 0.8858428405261162


Epoch:  80%|████████  | 4/5 [1:32:55<23:15, 1395.57s/it]

Average train loss: 0.7890534037939229


Epoch: 100%|██████████| 5/5 [1:56:13<00:00, 1394.68s/it]

Average train loss: 0.712100253040033


In [ ]:
from tqdm import tqdm, trange

model.train()
EPOCHS = 1
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    predictions, true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

Epoch: 100%|██████████| 1/1 [23:27<00:00, 1407.19s/it]

Average train loss: 0.6578933846337437


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/models/epo5_pythia_160md.pt')

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(plm,
                   bos_token_id=tokenizer.bos_token_id,
                   eos_token_id=tokenizer.eos_token_id,
                   pad_token_id=tokenizer.pad_token_id,
                   sep_token_id=tokenizer.sep_token_id,
                   output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision='step3000', config=config)
model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/911M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [ ]:
from torch.optim import AdamW
import torch
from peft import LoraConfig, TaskType
from peft import get_peft_model

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = get_peft_model(model, peft_config)

optimizer = AdamW(model.parameters(), lr=5e-5)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.print_trainable_parameters()
model.to(device)

trainable params: 294,912 || all params: 162,580,992 || trainable%: 0.18139389874063508


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50280, 768)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-11): 12 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/models/epo5_pythia_160md.pt'))

<All keys matched successfully>

In [ ]:
EPOCHS = 5

In [ ]:
from tqdm import tqdm, trange

model.train()
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    predictions, true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, attention_mask=masks, labels=labels)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

Epoch:  20%|██        | 1/5 [14:47<59:09, 887.27s/it]

Average train loss: 0.5638690618371781


Epoch:  40%|████      | 2/5 [29:34<44:22, 887.48s/it]

Average train loss: 0.5514546086039727


Epoch:  60%|██████    | 3/5 [44:19<29:32, 886.08s/it]

Average train loss: 0.5466434008166227


Epoch:  80%|████████  | 4/5 [59:04<14:45, 885.82s/it]

Average train loss: 0.5439496700333287


Epoch: 100%|██████████| 5/5 [1:13:48<00:00, 885.76s/it]

Average train loss: 0.5422154656138565


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/models/peft5_epo6_pythia_160md.pt')

In [ ]:
import torch
model.load_state_dict(torch.load('/content/drive/MyDrive/models/peft5_epo6_pythia_160md.pt'))

<All keys matched successfully>

In [ ]:
model = model.merge_and_unload()

In [ ]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset("csv", data_files="/content/drive/MyDrive/AICUP/opendid_test.tsv", delimiter='\t',
                          features = Features({
                              'fid': Value('string'),
                              'idx': Value('int64'),
                              'content': Value('string')}),
                              column_names=['fid', 'idx', 'content'])
valid_list = list(valid_data['train'])
print(len(valid_list))
valid_list

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

78980


[{'fid': '1097', 'idx': 1, 'content': '433475.RDC'},
 {'fid': '1097', 'idx': 12, 'content': 'Timmins, ELDEN'},
 {'fid': '1097', 'idx': 27, 'content': '43J47561,43J47561'},
 {'fid': '1097', 'idx': 46, 'content': 'Last edited : 7/9/2063  Page: 2'},
 {'fid': '1097', 'idx': 78, 'content': 'CLINICAL:'},
 {'fid': '1097',
  'idx': 88,
  'content': 'Metastatic cancer ?colorectal primary.'},
 {'fid': '1097', 'idx': 128, 'content': 'MACROSCOPIC:'},
 {'fid': '1097',
  'idx': 141,
  'content': 'Specimen labelled "Omentum secondary", consists of a piece of omentum 120 x 100 x 30mm.'},
 {'fid': '1097',
  'idx': 230,
  'content': 'On sectioning there are multiple fibrotic white ill-defined nodules identified.'},
 {'fid': '1097',
  'idx': 312,
  'content': 'Blocks: 1 to 5 - representative sections from the nodules.'},
 {'fid': '1097', 'idx': 371, 'content': 'Block 3 reserved block.'},
 {'fid': '1097', 'idx': 395, 'content': '(IC/vo 5.9.63)'},
 {'fid': '1097',
  'idx': 410,
  'content': 'MICROSCOPIC (R

In [ ]:
from torch.nn import functional as F
import re
import torch

dic = [ 'PATIENT', 'DOCTOR', 'USERNAME', 'PROFESSION', 'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER', 'AGE', 'DATE',
    'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDR', 'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT', 'LICENSE', 'VECHICLE', 'DEVICE', 'BIOID', 'IDNUM', 'OTHER']

du = r'P\d\d?(\.\d\d?)?W|P\d\d?(\.\d\d?)?K|P\d\d?(\.\d\d?)?Y|P\d\d?(\.\d\d?)?D|P\d\d?(\.\d\d?)?M'

def predict(model, tokenizer, input, max_new_tokens=400):

    def mondify(string, file, start, input):
        nonlocal end

        def found_position(string, found, start):
            nonlocal end
            position = string.find(found, end)

            if position != -1:
                end = position + len(found)
                res = f'{start+position}\t{start+position+len(found)}'
                return res

        pattern = re.compile(r'(\w+)\t(.+)')
        content = re.sub(pattern, r'\2', string)
        PHI = re.sub(pattern, r'\1', string)

        if PHI not in dic:
            return ""

        if PHI == 'DATE':
            cont = content.split('\t')
            start_end = found_position(input, cont[0], int(start))
            if len(cont) >= 2:
                if len(cont[1]) > 10:
                    cont[1] = cont[1][:10]
                content = cont[0] + '\t' + cont[1]
            else:
              return ""

        elif PHI == 'TIME':
            cont = content.split('\t')
            start_end = found_position(input, cont[0], int(start))
            if len(cont) >= 2:
                if len(cont[1]) > 16:
                    cont[1] = cont[1][:16]
                content = cont[0] + '\t' + cont[1]
            else:
              return ""
        elif PHI == 'DURATION':
            cont = content.split('\t')
            start_end = found_position(input, cont[0], int(start))
            if len(cont) >= 2:
                if len(cont[1]) > 4:
                    cont[1] = cont[1][:4]
                if not re.match(du, cont[1]):
                  return ""
                content = cont[0] + '\t' + re.match(du, cont[1]).group()
            else:
              return ""

        else:
            start_end = found_position(input, content, int(start))

        if start_end is None:
            return ""

        modified = re.sub(pattern, r'{}\t\1\t{}\t\2'.format(file, start_end), PHI + '\t' + content)
        return modified

    model.eval()
    input = input.split('<-#-#-#->')
    prompt = bos + input[2] + sep
    end = 0

    tks_info = tokenizer(prompt)
    text = tks_info['input_ids']

    inputs, past_key_values = torch.tensor([text]), None
    outputs = ''

    with torch.no_grad():
        for _ in range(max_new_tokens):
            out = model(inputs.to(device), past_key_values=past_key_values)
            logits = out.logits
            past_key_values = out.past_key_values
            log_probs = F.softmax(logits[:, -1], dim=-1)
            inputs = torch.argmax(log_probs, 1).unsqueeze(0)

            if tokenizer.decode(inputs.item()) == eos:
                break
            text.append(inputs.item())

        pred = tokenizer.decode(text)
        pred = pred[pred.find(sep) + len(sep):].replace(pad, "").replace(eos, "").strip()

        if pred.split('\t')[-1] == 'PHI: NULL':
            return ""

        labels = pred.split('\n')

        for i in range(len(labels)):
            if i < len(labels) - 1:
                if len(labels[i].split("\t")) < 2:
                    continue
                o = mondify(labels[i], input[0], input[1], input[2])
                if o == "":
                    continue
                outputs += o
                if end > len(input[2]):
                    break
                outputs += '\n'
            else:
                if len(labels[i].split("\t")) < 2:
                    continue
                o = mondify(labels[i], input[0], input[1], input[2])
                if o == "":
                    continue
                outputs += o

        outputs = outputs.rstrip('\n')

    return outputs


In [ ]:
ans = predict(model, tokenizer, '111'+'<-#-#-#->'+'222'+'<-#-#-#->'+'A family history should betaken.  Some patients with hepatolithiasis develop stones because of low levels of MDR3 (ABCB4 mutation). A family history of the following: early cholecystitis (low phospholipid-associated cholecystitis - LPAC - syndrome),jaundice of pregnancy in females, drug induced jaundice and unexplained biliary cirrhosis can be present in patients with inherited MDR3 deficiency.  If the family history is suggestive, ursodeoxycholic acid is of benefit to prevent lithiasis. I am not aware of any Australian centres that provided MDR3 gene analysis".  	')
print(ans)

In [ ]:
answer = []

lo = r'P\.?\ ?O\.?\ ?BOX \d+'
co = ['South Africa', 'Vietnam', 'USA', 'Australia']

for sent in valid_list:

  input_ = sent['content']
  if input_ is None: continue
  ans = predict(model, tokenizer, sent['fid']+'<-#-#-#->'+str(sent['idx'])+'<-#-#-#->'+input_)

  for c in co:
    if ans.find('ZIP') > -1: break
    pos1 = input_.find(c)
    if pos1 > -1:
      pos1 += sent['idx']
      if ans is None or ans == "":
        ans = sent['fid']+'\t'+'COUNTRY'+'\t'+str(pos1)+'\t'+str(pos1+len(c))+'\t'+c
        break
      a1 = sent['idx']
      a2 = ans.split('\n')
      for i in range(len(a2)):
        if pos1+len(c) <= int(a2[i].split('\t')[2]) and pos1 >= a1:
          a2.insert(i, sent['fid']+'\t'+'COUNTRY'+'\t'+str(pos1)+'\t'+str(pos1+len(c))+'\t'+c)
          ans = '\n'.join(a2)
          break
        a1 = int(a2[i].split('\t')[3])
      a1 = int(a2[-1].split('\t')[3])
      if pos1 >= a1:
        print(a2)
        a2.append(sent['fid']+'\t'+'COUNTRY'+'\t'+str(pos1)+'\t'+str(pos1+len(c))+'\t'+c)
        ans = '\n'.join(a2)
        print(a2)
        break

      for j in range(len(a2)):
        if c == a2[j].split('\t')[-1]:
          a2[j] = sent['fid']+'\t'+'COUNTRY'+'\t'+str(pos1)+'\t'+str(pos1+len(c))+'\t'+c
          ans = '\n'.join(a2)
          break


  match = re.search(lo, input_)
  if match:
    loc = match.group()
    pos = input_.find(loc)+sent['idx']
    if ans is None or ans == "":
      ans = sent['fid']+'\t'+'LOCATION-OTHER'+'\t'+str(pos)+'\t'+str(pos+len(loc))+'\t'+loc
      print(ans)
      answer.append(str(ans)+'\n')
      continue
    a1 = sent['idx']
    a2 = ans.split('\n')
    for i in range(len(a2)):
      if pos+len(loc) <= int(a2[i].split('\t')[2]) and pos >= a1:
        a2.insert(i, sent['fid']+'\t'+'LOCATION-OTHER'+'\t'+str(pos)+'\t'+str(pos+len(loc))+'\t'+loc)
        ans = '\n'.join(a2)
        break
      a1 = int(a2[i].split('\t')[3])

  if ans is None or ans == "": continue
  print(ans)
  answer.append(str(ans)+'\n')

with open('/content/drive/MyDrive/AICUP/answer.txt', 'w') as file:
    file.writelines(answer)

串流輸出內容已截斷至最後 5000 行。
5335	PATIENT	76	82	Mcendy
5335	STATE	98	101	TAS
5335	ZIP	103	107	2578
5335	DATE	133	142	15/2/2016	2016-02-15
5335	DATE	162	172	19/08/2064	2064-08-19
5335	DEPARTMENT	193	203	Immunology
5335	DOCTOR	207	224	DECLAN Petropulos
5335	DOCTOR	267	274	HILARIO
5335	DATE	319	326	14/7/66	2066-07-14
5335	DOCTOR	581	593	H. Villatora
5335	DOCTOR	2771	2781	H. Willits
5335	DATE	5152	5159	21/8/64	2064-08-21
5335	DATE	5186	5193	14/7/66	2066-07-14
5335	DOCTOR	5255	5269	Sarge Bellaire
5335	DATE	5271	5278	12/7/66	2066-07-12
5335	DATE	5438	5445	14/7/66	2066-07-14
5338	IDNUM	18	28	11N3652522
5338	IDNUM	49	57	11N36525
5338	IDNUM	58	66	11N36525
5338	DOCTOR	1032	1042	F Strotman
5338	DOCTOR	2738	2740	FH
5338	DOCTOR	2768	2779	F Burzynski
5338	DATE	5854	5863	28/6/2066	2066-06-28
5338	DATE	6481	6489	4/7/2066	2066-07-04
5341	IDNUM	14	24	79J385300O
5341	MEDICALRECORD	25	36	7933853.CCO
5341	PATIENT	38	53	FUGLSANG, LOGAN
5341	IDNUM	64	72	79J38530
5341	STREET	73	87	South Railroad
5341	CITY	88	95	KEMPS